In [1]:
import os
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score
import hopsworks


import warnings
warnings.filterwarnings("ignore")

# 1. Read your Hopsworks API Key from a file or environment variable
#    For example, if stored in 'hopsworks-api-key.txt'
with open('../data/hopsworks-api-key.txt', 'r') as f:
    api_key = f.read().strip()

# 2. Set the environment variable for Hopsworks
os.environ["HOPSWORKS_API_KEY"] = api_key

project = hopsworks.login()
fs = project.get_feature_store()

# Retrieve feature groups for Clash Royale
player_stats_fg = fs.get_feature_group(
    name='clash_royale_onehotencoding',
    version=5,
)
game_events_fg = fs.get_feature_group(
    name='clash_royale_dataset_onehotencoding',
    version=5,
)

df = player_stats_fg.read()

2025-01-06 16:53:44,484 INFO: Initializing external client
2025-01-06 16:53:44,485 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-06 16:53:46,710 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1175700
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.55s) 


In [2]:
dfstats = player_stats_fg.read()
dfstats

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.55s) 


,player1,player2,deck1,deck2,result
0,Diwel26,Thanos28739,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1


In [3]:
dfdata = game_events_fg.read()
dfdata

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.66s) 


,player_name,player_name2,deck1,deck2,result
0,#PRG9UCG8C,#2J9JGLUJP,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1,#QY8Q9LGY,#2R0PJUPVU,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2,#URQJURR0,#YPG29JVR,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
3,#2V8G9RPQ8,#20GL9LPLC,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",1
4,#PUVYRQRC,#2JC9PQ2L,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
...,...,...,...,...,...
620,#YYRPRU82V,#8R2YU8LPV,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
621,#LRUU0L22,#2C9CJVYQ8,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
622,#8PVGJCRQR,#CY00CGYV,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
623,#2VGU0RU2,#82PJ0L0CR,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [12]:
'''
from hsml.model_schema import ModelSchema
from hsml.model import Model
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
from uuid import uuid4  # To generate a unique id for the model

# Generate a unique id for the model
model_id = str(uuid4())


# Merge datasets (if necessary, based on shared keys)
# Assuming player_name1/player_name2 align with player1/player2
data = pd.concat([dfstats, dfdata], axis=0).reset_index(drop=True)

# Feature engineering
# For simplicity, we concatenate the decks and drop non-numeric columns
data['deck_combined'] = data['deck1'] + data['deck2']
data = data.drop(columns=['player1', 'player2', 'deck1', 'deck2'])

# Prepare features and target
X = pd.DataFrame(data['deck_combined'].tolist())
y = data['result']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Save model
model_path = "clash_royale_model.pkl"
joblib.dump(model, model_path)

# Register model in Hopsworks
model_schema = ModelSchema(X_train, y_train)
model_metadata = Model(
    id=model_id,  # Provide the unique id
    name="clash_royale_model",
    description="Random Forest model for predicting Clash Royale game outcomes",
    input_example=X_test.iloc[:1],
    model_schema=model_schema
)

mr = project.get_model_registry()


# Create and register the model
model = mr.sklearn.create_model(
    name="clash_royale_model",
    description="Random Forest model for predicting Clash Royale game outcomes",
    #model_path=model_path,  # Path to the saved model artifact
    input_example=X_test.iloc[:1].to_dict()
    #schema=model_schema  # Include schema if applicable
)

print("Model successfully registered in Hopsworks!")
'''

Model Accuracy: 1.00
Model successfully registered in Hopsworks!


In [4]:
df_1 = dfdata.iloc[:500,:]
df_2 = dfdata.iloc[500:,:]

In [5]:
selected_features = game_events_fg.select(['deck1','deck2','result'])

In [6]:
feature_view = fs.get_or_create_feature_view(
    name='clashroyale_fv',
    description="deck features",
    version=3,
    labels=['result'],
    query=selected_features,
)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = feature_view.train_test_split(test_size=0.2)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.32s) 


In [8]:
X_train

,deck1,deck2
0,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
4,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
620,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
621,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
622,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
623,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
print(X_train.isna().sum())

deck1    0
deck2    0
dtype: int64


In [10]:
y_train

,result
0,1
1,1
2,1
3,1
4,1
...,...
620,1
621,1
622,1
623,1


In [11]:
# Example of flattening columns
deck_flat = pd.DataFrame(data=list(X_train['deck1']), index=X_train.index)
deck2_flat = pd.DataFrame(data=list(X_train['deck2']), index=X_train.index)

# Merge flattened columns into the dataset
X_train = pd.concat([X_train.drop(['deck1', 'deck2'], axis=1), deck_flat, deck2_flat], axis=1)
X_test = pd.concat([X_test.drop(['deck1', 'deck2'], axis=1), 
                    pd.DataFrame(data=list(X_test['deck1']), index=X_test.index), 
                    pd.DataFrame(data=list(X_test['deck2']), index=X_test.index)], axis=1)


In [12]:
print(X_train.dtypes)
print(y_train.dtypes)

0      int32
1      int32
2      int32
3      int32
4      int32
       ...  
176    int32
177    int32
178    int32
179    int32
180    int32
Length: 362, dtype: object
result    int64
dtype: object


In [13]:
# Drop unnecessary columns
#train_features = X_train.drop(['date', 'player_id'], axis=1)
#test_features = X_test.drop(['date', 'player_id'], axis=1)

# Create and train the XGBoost Regressor
#xgb_regressor = XGBRegressor()
xgb_regressor = XGBRegressor(enable_categorical=True)


xgb_regressor.fit(X_train, y_train)

# Predict game outcomes on the test set
y_pred = xgb_regressor.predict(X_test)

# Calculate and print metrics
mse = mean_squared_error(y_test.iloc[:, 0], y_pred)
print("MSE:", mse)

r2 = r2_score(y_test.iloc[:, 0], y_pred)
print("R squared:", r2)

# Save predictions to DataFrame
df = y_test
df['predicted_game_outcome'] = y_pred

# Create directory for model artifacts
model_dir = "clash_royale_model"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
images_dir = model_dir + "/images"
if not os.path.exists(images_dir):
    os.mkdir(images_dir)

# Visualize and save feature importance
plot_importance(xgb_regressor, max_num_features=4)
feature_importance_path = images_dir + "/feature_importance.png"
plt.savefig(feature_importance_path)
plt.show()

from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Define input and output schemas
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)
schema_dict = model_schema.to_dict()

# Save the XGBoost model
xgb_regressor.save_model(model_dir + "/model.json")
mr = project.get_model_registry()

# Register the model in the Hopsworks model registry
cr_model = mr.python.create_model(
    name="clash_royale_xgboost_model",
    metrics={"mse": mse, "r2": r2},
    model_schema=model_schema,
    input_example=X_test.sample().values,
    description="Clash Royale game outcome predictor",
)

# Save the model artifacts to the registry
cr_model.save(model_dir)


AttributeError: 'DataFrame' object has no attribute 'dtype'